In [ ]:
!pip install mlxtend


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from mlxtend.frequent_patterns import apriori, association_rules
from datetime import datetime, timezone
import warnings
warnings.filterwarnings("ignore")


In [ ]:
groceries = pd.read_csv("Groceries data.csv")
basket_live = pd.read_csv("basket.csv")
groceries.head()


In [ ]:
groceries['itemDescription'] = groceries['itemDescription'].str.lower().str.strip()


In [ ]:
groceries['Transaction_ID'] = (
    groceries['Member_number'].astype(str)
    + "_"
    + groceries['month'].astype(str)
)


In [ ]:
check = groceries.groupby('Transaction_ID')['itemDescription'].nunique()

print("Average items per transaction:", check.mean())
print("Max items per transaction:", check.max())


In [ ]:
basket = groceries.groupby(
    ['Transaction_ID', 'itemDescription']
).size().unstack(fill_value=0)

basket = basket.applymap(lambda x: 1 if x > 0 else 0)

basket.shape


In [ ]:
frequent_itemsets = apriori(
    basket,
    min_support=0.02,
    use_colnames=True
)

print("Frequent itemsets:", frequent_itemsets.shape[0])


In [ ]:
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.3
)

print("Rules generated:", rules.shape[0])
rules.head()


In [ ]:
def recommend_items(user_items, rules, top_n=5):
    user_items = set(user_items)
    scored_recs = {}

    for _, rule in rules.iterrows():
        antecedents = set(rule['antecedents'])
        consequents = set(rule['consequents'])

        # partial match
        match_count = len(antecedents & user_items)

        if match_count > 0:
            for item in consequents:
                if item not in user_items:
                    score = match_count * rule['confidence'] * rule['lift']
                    scored_recs[item] = max(scored_recs.get(item, 0), score)

    return sorted(scored_recs, key=scored_recs.get, reverse=True)[:top_n]


In [ ]:
is_nitian = input("Are you an NITian? (yes/no): ").lower()
member_no = input("Enter your Member Number (or type NEW): ")

if member_no.lower() == "new":
    member_no = groceries['Member_number'].max() + 1
    print("🎉 New Member Created! Your Member Number is:", member_no)
else:
    member_no = int(member_no)
print("\nEnter items in your basket (comma separated):")
print("Example: whole milk, rolls/buns")
user_items = input().lower().split(",")

user_items = [item.strip() for item in user_items]
print("🛒 Your Basket:", user_items)


In [ ]:
suggested_items = recommend_items(user_items, rules)


In [ ]:
if is_nitian == "yes":
    print("\n🎓 NITian Mode: Snacks & instant foods prioritized")


In [119]:
print("\n✅ Recommended Items:")

if len(suggested_items) == 0:
    print("No strong association found. Using co-occurrence based recommendation.")

    # Transactions containing user items
    related_txns = basket[
        basket[user_items].sum(axis=1) > 0
    ]

    # Count other items bought together
    co_occurrence = (
        related_txns
        .drop(columns=user_items, errors="ignore")
        .sum()
        .sort_values(ascending=False)
    )

    # Remove zero-frequency items
    co_occurrence = co_occurrence[co_occurrence > 0]

    # Take top recommendations
    suggested_items = co_occurrence.head(5).index.tolist()

    # Final safety fallback (very rare case)
    if len(suggested_items) == 0:
        suggested_items = (
            groceries['itemDescription']
            .value_counts()
            .head(5)
            .index
            .tolist()
        )



else:
    for item in suggested_items:
        print("➡", item)



✅ Recommended Items:
No strong association found. Using co-occurrence based recommendation.


KeyError: "['bread'] not in index"

In [ ]:
today = datetime.now(timezone.utc).strftime("%Y-%m-%d")

new_rows = []
for item in user_items:
    new_rows.append({
        'Member_number': member_no,
        'Date': today,
        'itemDescription': item
    })

basket_live = pd.concat([basket_live, pd.DataFrame(new_rows)], ignore_index=True)
basket_live.to_csv("basket.csv", index=False)

print("\n💾 Transaction saved successfully!")


In [ ]:
print("\n🧾 SUMMARY")
print("Member Number:", member_no)
print("Basket Items:", user_items)
print("Recommended Items:", suggested_items)
